In [ ]:
# /usr/bin/env python

import requests, sys, os, pandas as pd, numpy as np, re, gzip, itertools, json, csv, shutil, xmltodict, urllib.request
from Bio.Blast.Applications import NcbiblastpCommandline as blastp
from Bio.Blast.Applications import NcbimakeblastdbCommandline as makeblastdb
from multiprocessing import Pool
from ftplib import FTP
from Bio import SeqIO
import natsort as ns
from natsort import natsorted
from Bio import Align, AlignIO, SeqIO, Phylo
from Bio.Align.Applications import ClustalOmegaCommandline as clustalo
from Bio.Align import substitution_matrices, MultipleSeqAlignment as MSA, AlignInfo
from Bio.PDB import PDBParser as PDB, PDBIO
from itertools import combinations
from random import sample
import statistics as st
import urllib.parse
import urllib.request

cwd = os.getcwd()

# 1. SPECIES SELECTION

In [ ]:
# WORKING DIRECTORIES

if not os.path.exists(cwd + '/fa') and not os.path.exists(cwd + '/gtf') and not os.path.exists(cwd + '/main') and not os.path.exists(cwd + '/blast_queries') and not os.path.exists(cwd + '/tandem') and not os.path.exists(cwd + '/orthologues') and not os.path.exists(cwd + '/appris') and not os.path.exists(cwd + '/alignments') and not os.path.exists(cwd + '/uniprot'):
    os.mkdir(cwd + '/fa'), os.mkdir(cwd + '/gtf'), os.mkdir(cwd + '/main'), os.mkdir(cwd + '/blast_queries'), os.mkdir(cwd + '/tandem'), os.mkdir(cwd + '/appris'), os.mkdir(cwd + '/orthologues'), os.mkdir(cwd + '/alignments'), os.mkdir(cwd + '/uniprot')

# SPECIES LIST

assemblies = requests.get("http://rest.ensembl.org/info/species?", headers={"Content-Type": "application/json"}).json()
json.dump(assemblies, open('assemblies.txt', 'w'))

def getaxonomy(specie):
    """return class, order, taxid, publications for a give specie name"""
    
    try:
        
        eutils = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
        taxid = xmltodict.parse(requests.get(eutils + 'esearch.fcgi?db=taxonomy&term=' + specie).content)['eSearchResult']['IdList']['Id']
        pubs = xmltodict.parse(requests.get(eutils + 'esearch.fcgi?db=pubmed&term=' + specie).content)['eSearchResult']['Count']
        taxons = xmltodict.parse(requests.get(eutils + 'efetch.fcgi?db=taxonomy&id=' + taxid + '&retmode=xml&rettype=full').content)['TaxaSet']['Taxon']['LineageEx']['Taxon']

        scientific_name = [taxons[x]['ScientificName'] for x in range(len(taxons)) if taxons[x]['ScientificName'] in ['Sauropsida', 'Mammalia', 'Actinopteri']]
        orders = [taxons[x]['ScientificName'] for x in range(len(taxons)) if taxons[x]['Rank'] == 'order']
        
        return scientific_name + orders + [specie.split('_', 1)[0]] + [specie.split('_', 1)[1]] + [taxid] + [pubs]
    
    except:
        pass

available_species = []
for assembly in assemblies['species']:
    taxonomy = getaxonomy(assembly['name'].capitalize())
    if not taxonomy == None and len(taxonomy) == 6:
        available_species.append(taxonomy)
        

species_list_df = pd.DataFrame(available_species)
species_list_df[5] = species_list_df[5].astype(int)
species_list_df_uniquegenre = species_list_df.sort_values(5, ascending=False).groupby(2, sort=False).head(1)
species_list_df_cap = species_list_df_uniquegenre.groupby(1, sort=False).head(12).sort_values([0, 1])

for line in species_list_df_cap.values.tolist():
    print(*line, sep=' ', file=open('species_list.txt', 'a'))

# 2. DOWNLOAD .faa AND .gtf

In [ ]:
# DOWNLOAD

species = [line.split()[2] + '_' + line.split()[3] for line in open('species_list.txt')]
queries = ['Homo_sapiens', 'Gallus_gallus', 'Danio_rerio']

assemblies = requests.get("http://rest.ensembl.org/info/species?", headers={"Content-Type": "application/json"}).json()
release = [line['release'] for line in assemblies['species'] if line['name'] == 'homo_sapiens']

def getassembly(specie):
    """return assembly identifier for a given species name"""
    
    for assembly in assemblies['species']:
        if specie.lower() == assembly['name']:
            return assembly['assembly']
        
def getassembly(specie):
    """return assembly identifier for a given species name"""
    
    for assembly in assemblies['species']:
        if specie.lower() == assembly['name']:
            return assembly['assembly']
        
def download(specie):
    """download faa from ensembl ftp"""
    
    ftp = FTP('ftp.ensembl.org')
    ftp.login()
    ftp.cwd('/pub/current_fasta/' + specie.lower() + '/pep/')
    ftp.retrbinary('RETR ' + specie + '.' + getassembly(specie) + '.pep.all.fa.gz', open(cwd + '/fa/' + specie + '.fa.gz', 'wb').write)
    if specie in queries:
        ftp.cwd('/pub/current_gtf/' + specie.lower() + '/')
        ftp.retrbinary("RETR " + specie + '.' + getassembly(specie) + '.' + str(release[0]) + '.gtf.gz', open(cwd + '/gtf/' + specie + '.gtf.gz', 'wb').write)
    ftp.quit()
        
if __name__ == '__main__':
    pool = Pool(processes=20)
    pool.map(download, [specie for specie in species])

# 3. MAIN ISOFORMS AND BLASTP

In [ ]:
# MAIN ISOFORMS AND BLASTP

species = [line.split()[2] + '_' + line.split()[3] for line in open('species_list.txt')]
queries = ['Homo_sapiens', 'Gallus_gallus', 'Danio_rerio']

assemblies = requests.get("http://rest.ensembl.org/info/species?", headers={"Content-Type": "application/json"}).json()

def getassembly(specie):
    """return assembly identifier for a given species name"""
    
    for assembly in assemblies['species']:
        if specie.lower() == assembly['name']:
            return assembly['assembly']

for line in queries:
    
    appris = pd.read_table(urllib.request.urlopen("http://apprisws.bioinfo.cnio.es/pub/current_release/datafiles/" + line.lower() + "/" + getassembly(line.lower()) + "/appris_data.principal.txt"), header=None)
    appris = appris[appris[4].str.contains('PRINCIPAL')].sort_values([1, 4]).drop_duplicates(1)
    for line2 in appris.values.tolist():
        print(*line2, sep='\t', file=open(cwd + '/appris/' + line + '_principal_appris.txt', 'a'))
    main_isoforms = appris[2].values.tolist()

    for fasta in SeqIO.parse(gzip.open(cwd + '/fa/' + line + '.fa.gz', 'rt'), 'fasta'):
        if 'gene_biotype:protein_coding' in fasta.description and re.split('\s', fasta.description)[4].split('transcript:')[1].split('.')[0] in main_isoforms:
            print('>' + fasta.id + '\n' + fasta.seq, file=open(cwd + '/main/' + line + '_main.fa', 'a'))

    gtf = pd.read_table(cwd + '/gtf/' + line + '.gtf.gz', compression='gzip', comment='#', header=None)
    gtf = gtf.join(gtf[8].str.split('"', expand=True).add_prefix('sec'))           
    gtf = gtf[gtf['sec5'].isin(main_isoforms)]

    main = []
    for line3 in gtf.values.tolist():
        if 'CDS' in line3[2]:
            chromosome, strand, gene_id, transcript_id, gene_name = line3[0], line3[6], line3[10], line3[14], line3[20]
            protein_id = [line for line in line3 if str(line).startswith(gene_id.split('0')[0][:-1] + 'P')]
        elif 'start' in line3[2]:
            start = line3[3]
        elif 'stop' in line3[2]:
            stop = line3[4]
            main.append([chromosome, gene_id, strand, protein_id[0], start, stop, transcript_id, gene_name])

    tsv = pd.DataFrame(main, columns=['Chromosome', 'Gene_id', 'Strand', 'Protein_id', 'Start', 'Stop', 'Transcript_id', 'Gene_name'])
    tsv['Chromosome'] = pd.Categorical(tsv['Chromosome'], ordered=True, categories=ns.natsorted(tsv['Chromosome'].unique()))
    tsv['Mean'] = tsv[['Start', 'Stop']].mean(axis=1)
    tsv = tsv.sort_values(['Chromosome', 'Mean'], ascending=(True, True)).drop(columns='Mean').to_csv((cwd + '/main/' + line + '.tsv'), index=False, sep='\t')
    
    makeblastdb(dbtype='prot', input_file=(cwd + '/main/' + line + '_main.fa'))()
    blastp(query=(cwd + '/main/' + line + '_main.fa'), db=(cwd + '/main/' + line + '_main.fa'), num_threads=20, evalue='10e-6', max_hsps=1, outfmt=7, max_target_seqs=5, out=cwd + '/blast_queries/' + line + '_blast.txt')()

# 4. DUPLICATIONS

In [ ]:
# DUPLICATIONS

for file in os.listdir(cwd + '/blast_queries'):
    if file.endswith('blast.txt'):

        x = pd.read_table(cwd + '/blast_queries/' + file, comment='#', header=None, names=['Query', 'Subject', 'Identity', 'A.lenght', 'Mismatches', 'Gap', 'Q.start', 'Q.end', 'S.start', 'S.end', 'Evalue', 'Bits'])
        x = x[x['Identity'] != 100].drop_duplicates('Query')
        x['Query'] = x['Query'].apply(lambda x: x.split('.')[0])
        x['Subject'] = x['Subject'].apply(lambda x: x.split('.')[0])
        columnab = [[line[0], line[1]] for line in x.values.tolist()]
        columnabbrh = [line for line in [[line[0], line[1]] for line in x.values.tolist()] if [line[1], line[0]] in columnab]
        xbrhdf = pd.merge(x, pd.DataFrame(columnabbrh), left_on=['Query', 'Subject'], right_on=[0, 1]).drop(columns=0)
        xy = pd.merge(pd.read_table(cwd + '/main/' + file.split('_blast')[0] + '.tsv'), xbrhdf, right_on='Query', left_on='Protein_id', how='outer').drop(columns=[1, 'Query'])
        xy = xy.values.tolist()

        tandem, convergent, divergent = [], [], []
        for x in range(len(xy)-1):
            strand, start, stop = xy[x][2], xy[x][4], xy[x][5]
            strand2, start2, stop2 = xy[x+1][2], xy[x+1][4], xy[x+1][5]
            query, subject = xy[x][3], xy[x+1][8]
            if query == subject:
                if strand == strand2:
                    if (strand == '+' and stop < start2) or (strand == '-' and start < stop2):
                        tandem.append([xy[x][1], xy[x][3], xy[x+1][1], xy[x+1][3], xy[x][9], xy[x][17]])
                if strand != strand2:
                    if strand == '+' and stop < stop2:
                        convergent.append([xy[x][1], xy[x][3], xy[x+1][1], xy[x+1][3], xy[x][9], xy[x][17]])
                    if strand == '-' and start < start2:
                        divergent.append([xy[x][1], xy[x][3], xy[x+1][1], xy[x+1][3], xy[x][9], xy[x][17]])
                        
        for line in tandem:
            print(*line, sep='\t', file=open(cwd + '/tandem/' + file.split('_blast')[0] + '_tandem.tsv', 'a'))
        for line in divergent:
            print(*line, sep='\t', file=open(cwd + '/tandem/' + file.split('_blast')[0] + '_divergent.tsv', 'a'))
        for line in convergent:
            print(*line, sep='\t', file=open(cwd + '/tandem/' + file.split('_blast')[0] + '_convergent.tsv', 'a'))

# 5. ORTHOLOGUES (COMPARA)

In [ ]:
# COMPARA ORTHOLOGUES

def getorthoss(line):
    """return a column of orthologues for a given query"""
    
    def getorthoss_letter(query, count, letter, specie, types):

        try:

            r = requests.get("https://rest.ensembl.org/homology/id/" + query + "?type=orthologues;aligned=0;cigar_line=0;compara=vertebrates", headers={"Content-Type": "application/json"})
            ortho = r.json()

            for x in range(len(ortho['data'][0]['homologies'])):
                target = ortho['data'][0]['homologies'][x]['target']
                per, pid, gid, spe = target['perc_id'], target['protein_id'], target['id'], target['species']
                source = ortho['data'][0]['homologies'][x]['source']
                per2, pid2, gid2, spe2 = source['perc_id'], source['protein_id'], source['id'], source['species']
                print(pid, gid, spe.capitalize(), count, letter, per, specie, str(count) + letter, file=open(cwd + '/orthologues/' + specie + '_' + types + '_orthologues.txt', 'a'))
            print(pid2, gid2, spe2.capitalize(), count, letter, 100, specie, str(count) + letter, file=open(cwd + '/orthologues/' + specie + '_' + types + '_orthologues.txt', 'a'))

        except:
            print(query, 'no_data', 'no_data', count, letter, 0, specie, str(count) + letter, file=open(cwd + '/orthologues/' + specie + '_' + types + '_orthologues.txt', 'a'))
    
    getorthoss_letter(line.split()[0], counter.get(line.split()[0]), 'A', line.split()[6].capitalize(), line.split()[7])
    getorthoss_letter(line.split()[2], counter.get(line.split()[2]), 'B', line.split()[6].capitalize(), line.split()[7])
       
        
counter = {} # assign a count for each pairs based on genomes
for file in os.listdir(cwd + '/tandem'):
    species = file.split('_')[0] + '_' + file.split('_')[1]
    types = file.split('_')[2].split('.')[0]
        
    for n, line in enumerate(open(cwd + '/tandem/' + file)):
        counter.update({line.split()[0]: n}), counter.update({line.split()[2]: n})
        
    #for line in open(cwd + '/tandem/' + file):
    #    getorthoss(line + '\t' + species + '\t' + types)

    if __name__ == '__main__':
        pool = Pool(processes=4)
        pool.map(getorthoss, [line + '\t' + species + '\t' + types for line in open(cwd + '/tandem/' + file)])

def orthologues_df(kind):

    # creo un dizionario con la tassonomia a partire dalla lista delle specie (specie, ordine, classe)
    classes, orders = {}, {}
    for line in open('species_list.txt'):
        classes.update({line.split()[2] + '_' + line.split()[3]: line.split()[0]})
        orders.update({line.split()[2] + '_' + line.split()[3]: line.split()[1]})

    # apro le tre tabelle
    h = pd.read_table(cwd + '/orthologues/Homo_sapiens_' + kind + '_orthologues.txt', sep='\s', engine='python', header=None)
    d = pd.read_table(cwd + '/orthologues/Danio_rerio_' + kind + '_orthologues.txt', sep='\s', engine='python', header=None)
    g = pd.read_table(cwd + '/orthologues/Gallus_gallus_' + kind + '_orthologues.txt', sep='\s', engine='python', header=None)

    # aggiungo lettere corrispondenti alle specie queries
    h[7] = 'H.' + h[7]
    d[7] = 'D.' + d[7]
    g[7] = 'G.' + g[7]

    # concateno le tre tabelle una dietro l'altra
    nobrh = pd.concat([h,d,g])

    # aggiungo voci della tassonomia
    nobrh[8] = nobrh[2].apply(lambda x: classes.get(x))
    nobrh[9] = nobrh[2].apply(lambda x: orders.get(x))
    nobrh = nobrh[(nobrh[8] == 'Mammalia') | (nobrh[8] == 'Sauropsida') | (nobrh[8] == 'Actinopteri')]
    nobrh[10] = nobrh[7].apply(lambda x: re.split('\.', x)[0])
    nobrh = nobrh.dropna(subset=[8])

    # numeriamo le hit di compara per ogni query per ogni specie (one-to-many)
    # se con gene di gallus da 20 ortologhi, numera questi ortologhi da 1 a 20
    nobrhsort = nobrh.sort_values([10, 3, 4, 2, 5], ascending=(False, True, True, True, False))
    nobrhsort[11] = nobrhsort.groupby([2, 7]).cumcount()+1

    # drop duplicati stessa percentuale d'identità per ogni query per ogni specie per il sorting
    nobrhsort = nobrhsort.drop_duplicates([2, 5, 7])

    # sorting per percentuale d'identità globale, drop geni duplicati tenendo il primo (one-to-one), drop stessa specie stesso gruppo 
    # "best-hit"
    bh = nobrhsort.sort_values(5, ascending=False).drop_duplicates(1).drop_duplicates([2,7])

    # tiene solo le prime hit di one-to-one
    # come nei casi in cui da ortologo come prima hit quando in realtà era il terzo
    # "reciprocal"
    brh = bh[bh[11] == 1]
    brh = brh.sort_values([10, 3, 4], ascending=(False, True, True))
    brh[20] = brh.groupby(7, sort=False)[7].transform("count")

    counts = brh[[7, 20]].drop_duplicates(7).values.tolist()
    couple_high_then3 = []
    for x in range(len(counts)-1):
        if counts[x][1] and counts[x+1][1] > 3 and re.split('A|B', counts[x][0])[0].split('.')[1] == re.split('A|B', counts[x+1][0])[0].split('.')[1]:
            couple_high_then3.append(counts[x][0])
            couple_high_then3.append(counts[x+1][0])
    brhfull = brh[brh[7].isin(couple_high_then3)]
    brhfull = brhfull.reset_index().drop(columns=['index'])
    brhfull[10] = brhfull.groupby([3,6], sort=False, as_index=False).ngroup()
    brhfull[10] = brhfull[10].astype(str) + brhfull[4]

    # pivot table
    brhfullpivot = brhfull.pivot(index=2, columns=10, values=0)
    brhfullpivot = brhfullpivot.reset_index()
    brhfullpivot['Classes'] = brhfullpivot[2].apply(lambda x: classes.get(x))
    brhfullpivot['Orders'] = brhfullpivot[2].apply(lambda x: orders.get(x))
    brhfullpivot = brhfullpivot.rename(columns={2: 'Species'})
    brhfullpivot = brhfullpivot.set_index(['Classes', 'Orders', 'Species']).sort_index()
    brhfullpivot = brhfullpivot.reindex(natsorted(brhfullpivot.columns), axis=1)
    brhfullpivot.to_csv(cwd + '/orthologues/' + kind + '_orthologues.csv', sep=';')
    
orthologues_df('tandem')
orthologues_df('divergent')
orthologues_df('convergent')

# 6. DATABASE

In [ ]:
# DATABASE

tandem_orthologs = pd.read_table(cwd + '/orthologues/' + 'tandem_orthologues.csv', sep=';').set_index(['Classes', 'Orders'])
convergent_orthologs = pd.read_table(cwd + '/orthologues/' + 'convergent_orthologues.csv', sep=';').set_index(['Classes', 'Orders'])
divergent_orthologs = pd.read_table(cwd + '/orthologues/' + 'divergent_orthologues.csv', sep=';').set_index(['Classes', 'Orders'])
orthologs = tandem_orthologs.values.tolist() + divergent_orthologs.values.tolist() + convergent_orthologs.values.tolist()

data = {}
for x in range(len(orthologs)):
    specie = orthologs[x][0] 
    
    main = {}
    for fasta in SeqIO.parse(gzip.open(cwd + '/fa/' + specie + '.fa.gz', 'rt'), 'fasta'):

        splitted = re.split('\s', fasta.description)
        pid = splitted[0].split('.')[0]
        gid = splitted[3].split('gene:')[1].split('.')[0]
        tra = splitted[4].split('transcript:')[1].split('.')[0]
        chrom = splitted[2].split(':')[2]
        strand = splitted[2].split(':')[5]
        if strand == '1':
            sta, sto = splitted[2].split(':')[3], splitted[2].split(':')[4]
        elif strand == '-1':
            sta, sto = splitted[2].split(':')[4], splitted[2].split(':')[3]
        try:
            symbol = splitted[7].split('gene_symbol:')[1]
        except:
            symbol = 'no_data'
        try:
            if '[Source' in fasta.description:
                product = re.split('description:', fasta.description)[1].split(' [Source')[0]
            else:
                product = re.split('description:', fasta.description)[1]
        except:
            product = 'no_data'

        main.update({pid: [str(fasta.seq), gid, chrom, strand, int(sta), int(sto), len(fasta.seq), product.strip(), symbol, specie]})

    for line in orthologs[x]:
        if not 'nan' in str(line):
            data.update({line: main.get(line)})
                
json.dump(data, open('database.json', 'w')) 

# 7. FASTAS FOR ALIGNMENTS

In [ ]:
# FASTA FOR ALIGNMENTS

database = json.load(open('database.json'))

if not os.path.exists(cwd + '/alignments/tandem_alignments') and not os.path.exists(cwd + '/alignments/convergent_alignments') and not os.path.exists(cwd + '/alignments/divergent_alignments'):
    os.mkdir(cwd + '/alignments/tandem_alignments'), os.mkdir(cwd + '/alignments/convergent_alignments'), os.mkdir(cwd + '/alignments/divergent_alignments')
    
def fastaforaligments(kind):
    data = pd.read_csv(cwd + '/orthologues/' + kind + '_orthologues.csv', sep=';').set_index(['Classes', 'Orders', 'Species'])
    pairs_ab = list(zip([line for line in data.columns.tolist() if 'A' in line], [line for line in data.columns.tolist() if 'B' in line]))

    for x in range(len(pairs_ab)):
        for f in data[list(pairs_ab[x])].values.tolist():

            def getfastas(symbol, product, lenght, gene, specie, tandem, pid, fasta):
                    print('>' + pid + '\t' + gene + '\t' + lenght + '\t' + symbol + '\t' + product + '\t' + specie + '\t' + tandem + '\n' + fasta, file=open(cwd + '/alignments/' + kind + '_alignments' + '/' + re.split('A|B', tandem)[0] + '.fa', 'a'))            

            if not 'nan' in str(f[0]): 
                getfastas(database.get(f[0])[8], database.get(f[0])[7], 
                          str(len(database.get(f[0])[0])), database.get(f[0])[1], 
                          database.get(f[0])[9], data[list(pairs_ab[x])].columns.tolist()[0], 
                          f[0], database.get(f[0])[0])
                
            if not 'nan' in str(f[1]):
                getfastas(database.get(f[1])[8], database.get(f[1])[7], 
                          str(len(database.get(f[1])[0])), database.get(f[1])[1], 
                          database.get(f[1])[9], data[list(pairs_ab[x])].columns.tolist()[1], 
                          f[1], database.get(f[1])[0])
            
fastaforaligments('tandem')
fastaforaligments('convergent')
fastaforaligments('divergent')

# 8. ALIGNMENTS (CLUSTALO + SCORING)

In [ ]:
# ALIGNMENTS

def conservation(msa,matrix): 
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load(matrix)
    numbers, columns, scores = [], [], []
    for n in range(msa.get_alignment_length()):
        columns.append((msa[:,n]))
        numbers.append(n+1)
    for c in columns:
        c = list(c)
        if 'X' in c:
            c.remove('X')
        pairs = list(combinations(c,2))
        score = []
        try:
            for p in pairs:
                if not '-' in p:
                    score.append(aligner.score(p[0],p[1]))
            scores.append(sum(score)/len(pairs))
        except:
            pass
    return (list(zip(numbers,columns,scores)))

def getalignmentjson(folder):
    """return similarity, number of sequences, alignment lenght, position etc.. for a given aligned fasta file"""

    alignments_log = {}

    # define filename    
    number_of_files = max([int(file.split('.')[0]) for file in os.listdir(cwd + '/alignments/' + folder) if file.endswith('.fa')])
    for name in range(number_of_files):
        name = str(name)
        fa = cwd + '/alignments/' + folder + '/' + name + '.fa'
        fasta = cwd + '/alignments/' + folder + '/' +  name + '.fasta'
        matrix = 'BLOSUM62'

        len_A, len_B = [], []
        for f in list(SeqIO.parse(fa, 'fasta')):
            if 'A' in f.description.split('\t')[-1]:
                len_A.append(int(f.description.split('\t')[2]))
            elif 'B' in f.description.split('\t')[-1]:        
                len_B.append(int(f.description.split('\t')[2]))

        similarity = round(min(st.mean(len_A),st.mean(len_B))/max(st.mean(len_A),st.mean(len_B))*100,2)
        if similarity > 60:

            clustalo(infile = fa, outfile = fasta, force = False)()
            # sampling sequences from alignments
            alignment = AlignIO.read(fasta, 'fasta')

            dif_score, A, B, gap = [], [], [], []
            for a in alignment:
                if 'A' in a.description.split('\t')[-1]:
                    A.append(a) 
                elif 'B' in a.description.split('\t')[-1]:
                    B.append(a)
            A, B = sample(A, min(len(A),len(B))), sample(B, min(len(A),len(B)))
            aln_A, aln_B, aln = MSA(A), MSA(B), MSA(A+B)
            cons_A, cons_B = AlignInfo.SummaryInfo(aln_A).dumb_consensus(), AlignInfo.SummaryInfo(aln_B).dumb_consensus()

            differences = list(zip(conservation(aln,matrix), conservation(aln_A,matrix), conservation(aln_B,matrix)))   
            hit, res_A, res_B = [], [], []
            for d in differences:
                dif_score.append([d[0][0], d[1][1], d[2][1], round(min(d[1][2], d[2][2])-d[0][2],2)])
                hit.append(d[0][0])
                res_A.append(cons_A[d[0][0]-1])
                res_B.append(cons_B[d[0][0]-1])

            pos_A, pos_B, pos_C = [], [], []
            for n in range(len(alignment)):
                for res in zip(res_A,hit,res_B):
                    if 'ENSP0' in alignment[n].name:
                        try:
                            pos_C.append([res[0], res[2]])
                            if 'B' in alignment[n].description.split('\t')[-1][-1]:
                                pos_B.append(res[1]-list(alignment[n][0:res[1]-1]).count('-'))
                            else:
                                pos_B.append(None)
                            if 'A' in alignment[n].description.split('\t')[-1][-1]:
                                pos_A.append(res[1]-list(alignment[n][0:res[1]-1]).count('-'))
                            else:
                                pos_A.append(None)
                        except:
                            pass

            x = pd.concat([pd.Series(pos_A).dropna().reset_index().astype(np.int64), pd.Series(pos_B).dropna().reset_index().astype(np.int64)], axis=1).drop(columns='index').astype(str)
            y = zip(x.values.tolist(), pos_C)
            ratio = round(len(hit)/np.mean(min(len_A,len_B))*100, 2)

            positions = {}
            for p in list(zip(dif_score, y)):
                key = p[0][0]
                alignment_A = p[0][1]
                alignment_B = p[0][2]
                score = p[0][3]
                homo_position_A = p[1][0][0]
                homo_position_B = p[1][0][1]
                homo_residue_A = p[1][1][0]
                homo_residue_B = p[1][1][1]
                position = {'alignment_A': alignment_A, 'alignment_B': alignment_B, 'Score': score, 
                            'Homo_position_A': homo_position_A, 'Homo_position_B': homo_position_A, 
                            'Homo_residue_A': homo_residue_A, 'homo_residue_B': homo_residue_B}
                positions.update({key: position})

            accessions = []
            if any('ENSP0'  in string for string in [fasta.id for fasta in SeqIO.parse
                                                     (cwd + '/alignments/' + folder + '/' + name + '.fa', 'fasta')
                                                     if 'A' in fasta.description.split('\t')[6]]) == False:
                accessions.append(None)
            if any('ENSP0'  in string for string in [fasta.id for fasta in SeqIO.parse
                                                     (cwd + '/alignments/' + folder + '/' + name + '.fa', 'fasta')
                                                     if 'B' in fasta.description.split('\t')[6]]) == False:
                accessions.append(None)
            for fasta in SeqIO.parse(cwd + '/alignments/' + folder + '/' + name + '.fa', 'fasta'):
                if 'ENSP0' in fasta.id and 'A' in fasta.description.split('\t')[6]:
                    accessions.append(fasta.id) 
                if 'ENSP0' in fasta.id and 'B' in fasta.description.split('\t')[6]:
                    accessions.append(fasta.id) 


            alignments_log.update({name: {'Accessions': accessions, 'Similarity': similarity, 'Number_of_sequences': len(list(SeqIO.parse(fa, 'fasta'))), 'Alignment_lenght': aln.get_alignment_length(), 'Ratio': ratio, 'Positions': positions, 'Scores': [v['Score'] for k,v in positions.items()]}})


    json.dump(alignments_log, open(cwd + '/alignments/' + folder + '/alignments.json', 'w'))
    

getalignmentjson('tandem_alignments')
getalignmentjson('convergent_alignments')
getalignmentjson('divergent_alignments')

# 9. FEATURES (UNIPROT)

In [ ]:
def features(kind):

    # apro tabella ortologhi e creo lista degli accession di Homo da analizzare
    orthos = pd.read_table(cwd + '/orthologues/' + kind + '_orthologues.csv', sep=';').set_index(['Classes', 'Orders', 'Species'])
    homo = pd.DataFrame(orthos.loc[('Mammalia', 'Primates', 'Homo_sapiens')])
    homo_joined = ' '.join(map(str, [line[0] for line in homo.values.tolist()]))

    # preparo gli accession per la conversione Ensembl_ID --> Uniprot_ID
    params = {'from': 'ENSEMBL_PRO_ID', 'to': 'ACC', 'format': 'tab', 'query': homo_joined}
    url = 'https://www.uniprot.org/uploadlists/'
    req = urllib.request.Request(url, urllib.parse.urlencode(params).encode('utf-8'))
    with urllib.request.urlopen(req) as f:
        response = f.read()

    # scrivo un dizionario di conversione
    converted = {}
    for line in response.decode('utf-8').split('\n'):
        if not 'From' in line and not line.split() == []:
            converted.update({line.split()[1]: line.split()[0]})

    # recupero le features da uniprot in formato json
    def getfeatures(uniprot_id):
        """return the Uniprot complete dataset for a given Uniprot ID"""
        try:
            r = requests.get("https://www.ebi.ac.uk/proteins/api/proteins?size=-1&accession=" + uniprot_id, headers={ "Accept" : "application/json"})
            data = pd.json_normalize(r.json())
            return data
        except:
            return str(uniprot_id) + ' not_found'

    # concateno le features in formato tabulare e le scarico in locale
    appended_data = []
    for k, v in converted.items():
        data = getfeatures(k)
        appended_data.append(data)
    appended_data = pd.concat(appended_data)
    appended_data.to_csv(cwd + '/uniprot/' + kind + '_raw_features.csv')

    raw_features = pd.read_table(cwd + '/uniprot/' + kind + '_raw_features.csv', sep=',')
    raw_features = raw_features[raw_features['accession'].isin(list(converted.keys()))]

    # organizzo e filtro le features, unendole in un unico dataframe
    dataframe_list = []
    for line in raw_features[['accession', 'gene', 'protein.recommendedName.ecNumber', 'protein.recommendedName.fullName.value', 'features']].iterrows():
        if not str(line[1]['features']) == 'nan' and 'description' in list(eval(line[1]['features'])[0].keys()):
            dataframe = pd.DataFrame(eval(line[1]['features']))[['type', 'category', 'description', 'begin', 'end']]
            dataframe['accession'] = line[1]['accession']
            if not str(line[1]['gene']) == 'nan':
                dataframe['gene'] = eval(line[1]['gene'])[0]['name']['value']
            else:
                dataframe['gene'] = 'NaN'
            if not str(line[1]['protein.recommendedName.ecNumber']) == 'nan':
                dataframe['EC_number'] = eval(line[1]['protein.recommendedName.ecNumber'])[0]['value']
            else:
                dataframe['EC_number'] = 'NaN'
            dataframe['full_protein_name'] = line[1]['protein.recommendedName.fullName.value']
            dataframe['ensembl_accession'] = converted.get(line[1]['accession'])
            dataframe_list.append(dataframe)

    df = pd.concat(dataframe_list)[['accession', 'ensembl_accession', 'gene', 'full_protein_name', 'type', 'category', 'description', 'EC_number', 'begin', 'end']]

    orthos = pd.read_table(cwd + '/orthologues/' + kind + '_orthologues.csv', sep=';')
    orthos = list(zip(orthos[orthos['Species'] == 'Homo_sapiens'].values.tolist()[0][3:], orthos[orthos['Species'] == 'Homo_sapiens'].columns.tolist()[3:]))
    orthos = pd.DataFrame(orthos, columns = ['ensembl_accession', 'orthogroup'])
    df = pd.merge(df, orthos, on='ensembl_accession')
    df = df[['accession', 'orthogroup', 'ensembl_accession', 'gene', 'full_protein_name', 'type', 'category', 'description', 'EC_number', 'begin', 'end']]

    df.to_csv(cwd + '/uniprot/' + kind + '_filtered_features.csv')

    # scrivo i range di posizioni che mi interessano (in questo caso -1 e +1 sia per begin che per end)
    df = pd.read_table(cwd + '/uniprot/' + kind + '_filtered_features.csv', sep=',').drop(columns='Unnamed: 0').reset_index()
    df['begin'] = df['begin'].apply(lambda x: None if '~' in str(x) else x)
    df['end'] = df['end'].apply(lambda x: None if '~' in str(x) else x)
    df['b+1'] = df['begin'].apply(lambda x: str(int(x)+1) if not x == None else x)
    df['b-1'] = df['begin'].apply(lambda x: str(int(x)-1) if not x == None else x)
    df['e+1'] = df['end'].apply(lambda x: str(int(x)+1) if not x == None else x)
    df['e-1'] = df['end'].apply(lambda x: str(int(x)-1) if not x == None else x)
    df['b+1'], df['b-1'] = df['b+1'].astype(float), df['b-1'].astype(float)
    df['e+1'], df['e-1'] = df['e+1'].astype(float), df['e-1'].astype(float)
    df['begin'], df['end'] = df['begin'].astype(float), df['end'].astype(float)

    uniprot_list = df[['ensembl_accession', 'index', 'begin', 'b+1', 'b-1', 'end', 'e+1', 'e-1']].values.tolist()

    positions = json.load(open(cwd + '/alignments/' + kind + '_alignments/alignments.json'))

    # confronto le posizioni trovate in uniprot con quelle trovate mediante allineamenti
    prova = []

    for group in positions:
        x = positions[group]
        acc_A, acc_B = x['Accessions'][0], x['Accessions'][1]

        for pos in x['Positions']:
            x2 = x['Positions'][pos]
            score = x['Positions'][pos]['Score']

            if score > 1:
                al_A, al_B = x2['alignment_A'], x2['alignment_B']
                pos_A, pos_B = float(x2['Homo_position_A']), float(x2['Homo_position_B'])
                res_A, res_B = x2['Homo_residue_A'], x2['homo_residue_B']

                for line in uniprot_list:
                    accession = line[0]
                    index = line[1]
                    begin, begin_plus_one, begin_minus_one = line[2], line[3], line[4]
                    end, end_plus_one, end_minus_one = line[5], line[6], line[7]

                    if accession == acc_A:

                        if pos_A == begin or pos_A == begin_plus_one or pos_A == begin_minus_one or pos_A == end or pos_A == end_plus_one or pos_A == end_minus_one:
                            prova.append([acc_A, index, pos_A, res_A, res_B, al_A, al_B, score])
                        if pos_B == begin or pos_B == begin_plus_one or pos_B == begin_minus_one or pos_B == end or pos_B == end_plus_one or pos_B == end_minus_one:
                            prova.append([acc_A, index, pos_B, res_A, res_B, al_A, al_B, score])    

                    if accession == acc_B:
                        if pos_A == begin or pos_A == begin_plus_one or pos_A == begin_minus_one or pos_A == end or pos_A == end_plus_one or pos_A == end_minus_one:
                            prova.append([acc_B, index, pos_A, res_A, res_B, al_A, al_B, score])
                        if pos_B == begin or pos_B == begin_plus_one or pos_B == begin_minus_one or pos_B == end or pos_B == end_plus_one or pos_B == end_minus_one:
                            prova.append([acc_B, index, pos_B, res_A, res_B, al_A, al_B, score]) 

    positions_found = pd.DataFrame(prova, columns=['ensembl_accession', 'index', 'position_found', 'residue_A', 'residue_B', 'alignment_A', 'alignment_B', 'score'])

    final_df = pd.merge(df, positions_found, on=['index', 'ensembl_accession']).drop_duplicates()

    # recupero i full protein name di ensembl dal database (proteina in esame e partner)
    database = json.load(open('database.json'))
    orthos = pd.read_table(cwd + '/orthologues/' + kind + '_orthologues.csv', sep=';')
    orthos = list(zip(orthos[orthos['Species'] == 'Homo_sapiens'].values.tolist()[0][3:], orthos[orthos['Species'] == 'Homo_sapiens'].columns.tolist()[3:]))
    orthos = pd.DataFrame(orthos, columns = ['ensembl_accession', 'orthogroup'])
    final_df = pd.merge(final_df, orthos, on=['ensembl_accession', 'orthogroup'])
    orthos['full_name'] = orthos['ensembl_accession'].apply(lambda x: database.get(x)[7] if not database.get(x) == None else x)

    final_df['full_name_ensembl_partner'] = final_df['orthogroup'].apply(lambda x: database.get(
        orthos[orthos['orthogroup'] == re.split('A|B', x)[0] + {'A': 'B', 'B': 'A'}.get(x[-1])].values.tolist()[0][0])[7] if not database.get(
        orthos[orthos['orthogroup'] == re.split('A|B', x)[0] + {'A': 'B', 'B': 'A'}.get(x[-1])].values.tolist()[0][0]) == None else None)

    final_df['full_name_ensembl'] = final_df['orthogroup'].apply(lambda x: database.get(
        orthos[orthos['orthogroup'] == x].values.tolist()[0][0])[7] if not database.get(
        orthos[orthos['orthogroup'] == x].values.tolist()[0][0]) == None else None)

    # ordino la tabella e la salvo in locale
    final_df = final_df.drop(columns=['index', 'b+1', 'b-1', 'e+1', 'e-1']).drop_duplicates()[['orthogroup', 'accession', 'ensembl_accession', 'gene', 'full_protein_name', 'full_name_ensembl', 'full_name_ensembl_partner', 'type', 'category', 'description', 'begin', 'end', 'position_found', 'score', 'residue_A', 'residue_B', 'alignment_A', 'alignment_B', 'EC_number']]
    final_df = final_df[~final_df['category'].str.contains('STRUCTURAL')]
    final_df = final_df[~final_df['category'].str.contains('TOPOLOGY')]
    final_df.to_csv(cwd + '/uniprot/' + kind + '_features.csv')

features('Convergent')
features('Divergent')
features('Tandem')